In [1]:
# !python --version

In [2]:
# !pip install -q -U pip
# !pip install -q numpy
# !pip install -q pandas
# !pip install -q ckiptagger
# !pip install -q tqdm
# !pip install -q tensorflow==1.14.0
# !pip install -q ipywidgets

In [3]:
import pandas as pd
import numpy as np

from ckiptagger import data_utils, WS, POS
from tqdm.notebook import tqdm

In [4]:
df_train = pd.read_csv('news_clustering_train.tsv', sep='\t')
df_test = pd.read_csv('news_clustering_test.tsv', sep='\t')

In [5]:
df_train

,index,class,title
0,0,體育,亞洲杯奪冠賠率：日本、伊朗領銜 中國竟與泰國並列
1,1,體育,9輪4球本土射手僅次武磊 黃紫昌要搶最強U23頭銜
2,2,體育,如果今年勇士奪冠，下賽季詹姆斯何去何從？
3,3,體育,超級替補！科斯塔本賽季替補出場貢獻7次助攻
4,4,體育,騎士6天里發生了啥？從首輪搶七到次輪3-0猛龍
...,...,...,...
1795,1795,遊戲,LOL：麻辣香鍋韓服Rank不合成打野刀？電刀巨魔新套路連勝中
1796,1796,遊戲,穩住，我們能贏！因為我們擁有這種強大的力量
1797,1797,遊戲,騰訊是怎樣毀掉《鬥戰神》這款可能成神的作品的？
1798,1798,遊戲,LOL你不知道的黑科技打法！


In [6]:
count = 0 
for index, row in df_train.iterrows():
    if count == 5:
        break
    print(index, row)
    count += 1

0 index                           0
class                          體育
title    亞洲杯奪冠賠率：日本、伊朗領銜 中國竟與泰國並列
Name: 0, dtype: object
1 index                            1
class                           體育
title    9輪4球本土射手僅次武磊 黃紫昌要搶最強U23頭銜
Name: 1, dtype: object
2 index                       2
class                      體育
title    如果今年勇士奪冠，下賽季詹姆斯何去何從？
Name: 2, dtype: object
3 index                        3
class                       體育
title    超級替補！科斯塔本賽季替補出場貢獻7次助攻
Name: 3, dtype: object
4 index                          4
class                         體育
title    騎士6天里發生了啥？從首輪搶七到次輪3-0猛龍
Name: 4, dtype: object


In [7]:
train_titles = {row['index']: row['title'] for _, row in df_train.iterrows()}
train_classes = {row['index']: row['class'] for _, row in df_train.iterrows()}

test_titles = {row['index']: row['title'] for _, row in df_test.iterrows()}
test_classes = {row['index']: row['class'] for _, row in df_test.iterrows()}

In [8]:
all_news_class = ['體育', '財經', '科技', '旅遊', '農業', '遊戲']

# 斷詞 + POS

In [9]:
# data_utils.downlaod_data_gdown('./')
ws = WS('./data/')
pos = POS('./data/')

In [14]:
train_title_cuts = {}
for index, title in tqdm(train_titles.items()):
    # YOUR CODE HERE
    word_s = ws([title], sentence_segmentation=True, segment_delimiter_set={'?', '？', '!', '！', '。', ',', '，', ';', ':', '、'})
    word_p = pos(word_s)

    # END YOUR CODE
    train_title_cuts[index] = list(zip(word_s[0], word_p[0]))

In [15]:
train_title_cuts[0]

[('亞洲杯', 'Nb'),
 ('奪冠賠率', 'Na'),
 ('：', 'COLONCATEGORY'),
 ('日本', 'Nc'),
 ('、', 'PAUSECATEGORY'),
 ('伊朗', 'Nc'),
 ('領銜', 'VB'),
 (' ', 'WHITESPACE'),
 ('中國', 'Nc'),
 ('竟', 'D'),
 ('與', 'P'),
 ('泰國', 'Nc'),
 ('並列', 'VG')]

In [16]:
test_title_cuts = {}
for index, title in tqdm(test_titles.items()):
    # YOUR CODE HERE
    word_s = ws([title], sentence_segmentation=True, segment_delimiter_set={'?', '？', '!', '！', '。', ',', '，', ';', ':', '、'})
    word_p = pos(word_s)
    # END YOUR CODE
    test_title_cuts[index] = list(zip(word_s[0], word_p[0]))

In [17]:
train_title_cuts[120]

[('國腳', 'Na'),
 ('張呈棟', 'Nb'),
 ('：', 'COLONCATEGORY'),
 ('從', 'D'),
 ('沒', 'D'),
 ('想', 'VE'),
 ('過', 'Di'),
 ('自己', 'Nh'),
 ('會', 'D'),
 ('出', 'VC'),
 ('一', 'Neu'),
 ('本', 'Nf'),
 ('書', 'Na')]

In [11]:
# train_title_cuts[120]

[('國腳', 'Na'),
 ('張呈棟', 'Nb'),
 ('：', 'COLONCATEGORY'),
 ('從', 'D'),
 ('沒', 'D'),
 ('想', 'VE'),
 ('過', 'Di'),
 ('自己', 'Nh'),
 ('會', 'D'),
 ('出', 'VC'),
 ('一', 'Neu'),
 ('本', 'Nf'),
 ('書', 'Na')]

# Bag of Words (BOW)

In [18]:
word2index = {}
index2word = {}
# 產生字與index對應的關係
# YOUR CODE HERE
n = 0
for _, item in train_title_cuts.items():
    for word, pos in item:
        if word not in word2index:
            index2word[n] = word
            word2index[word] = n
            n += 1
        
# END YOUR CODE

In [19]:
word2index['溫暖']

1512

In [20]:
index2word[1520]

'許'

In [13]:
#word2index['溫暖']

1520

In [14]:
#index2word[1520]

'溫暖'

In [23]:
def get_bow_vector(pairs, word2index):
    # YOUR CODE HERE
    vector = np.zeros(len(word2index))
    for word, pos in pairs:
        vector[word2index[word]] += 1

    # END YOUR CODE
    return vector

In [24]:
get_bow_vector(train_title_cuts[120], word2index)

array([0., 0., 1., ..., 0., 0., 0.])

# 排除較無意義的詞性

In [25]:
pos_analysis = {}
for _, pairs in train_title_cuts.items():
    for word, flag in pairs:
        if flag not in pos_analysis:
            pos_analysis[flag] = set()
        pos_analysis[flag].add(word)

for flag, words in pos_analysis.items():
    print(flag, ':', list(words)[:100])
    print('=======================')

Nb : ['G7', '馬化騰', '雷軍', '張藝謀', '帕克成', '董明珠', '宇通', '尤文', '慧商', '嘉年華', '盼保', '張某', '綠心', '微博', '瓜迪奧拉', '王者榮耀', '韓信', '克羅托內 ', '伯德', '李秋平', '余小C', '宋清輝', '亞特拉斯賽', '阿里舜宇', '劉偉', '科斯塔', '張呈棟', '拉卡拉', 'NBA', 'U23', '朱嘯虎', '火箭隊', '英特爾', '馬里奧', '切沃', '周琦', '單薇恩', '朱婷', '男乒', '劉熾平', '堪比比薩', '王者榮', '朗多', '蓋倫', '安永', '恆大', '切爾西', '大豐', '瑞和寶', '阿里', '泰森', '米切爾西蒙斯', '阿珂', '衛報', '·瞻', '石川佳純', '淘寶', '里坦克', '卡拉黑', '金英權', '侯森', '萊萬', '拜仁續', '東皇', '群里', '康德', '林員', '裴擒虎', '泰山', '神武', '林斌', '斐濟', '寶二', '楊柳夏', '雅桑克萊', '娜迦王海伯倫', '陶華碧', '拳皇', '星巴克', '詹姆斯', '波爾', '王', '山東隊', '趙雲', '騰訊王', '武磊', '比格利亞', '力帆', '尤納斯', '施之皓', '泰拳王西', '龔方雄', 'V10', '昌圖', '桑德羅', '騎士隊', '權健', '威廉姆斯', '卡納瓦羅', '碧生源']
Na : ['身', '建築', '心理', ' 馬雲', '聯盟', '搏擊', '命運', '薪令', '櫻花', '自建房', '廠長', '大洋鏈', '排行榜', '王者榮耀', '姐', '總冠軍', '作物', '螃蟹', '梭', '滄桑', '公路', '情報', '大水', '獨角獸', '黑土', '鴨', '生物', '插秧期', '麥塊', '合規性', '損害', '成長股', '定制', '地確權', '空地', '慘案', '價格', '世錦賽', '門票', '馬哈魚', '地鐵', '可可', '客場', '巨頭', '單位', '好兄弟', '心意', '雨傘', '鬥戰神', '地圖', '東北

COMMACATEGORY : [',', '，']
Nes : ['近', '何', '頭', '上', '下', '另', '每', '某', '本', '各', '同', '後', '前', '貴', '首', '該']
QUESTIONCATEGORY : ['？']
VC : ['移', '送走', '繼承', '吃到', '指認', '指責', '打', '嫁', '發文', '列', '賒', '喜出', '治療', '養恩', '打野', '行', '追上', '吸收', '攻略', '搞', '堆上', '分擔', '叫出', '利用', '取代', '帶', '看待', '創新', '遇上', '擊', '影響', '收割', '換', '飲', '接', '挑', '清理', '發展', '套現', '推動', '堵', '迎來', '收', '加盟', '進出口', '戰釀', '引爆', '打造', '比較', '鋪開', '等到', '虧', '領取', '貢獻', '分析', '報', '結', '評價', '丟棄', '製造', '閃', '買錯', '催收', '取消', '更新', '削弱', '包裝', '零售', '募', '更換', '擔', '考', '藏', '戲耍', '誤觸', '引進', '整理', '採訪', '打開', '扣', '栽', '蒸', '唱', '打贏', '封禁', '踢', '重建', '割', '掛', '迎接', '秀', '探訪', '攢', '避開', '定', '花', '尋', '抽', '逼死', '登記']
EXCLAMATIONCATEGORY : ['！', '!']
Di : ['了', '起來', '個', '過', '著']
Nep : ['此', '哪', '這', '什麼', '啥', '那', '其中', '其']
Caa : ['比', '暨', '至', '跟', '又', '或是', '與', 'VS', '還是', '及', '或', '到', '和', '或者']
VCL : ['走訪', '前來', '入', '遠離', '到達', '栽進', '住', '往', '走過', '升至', '深入', '重回', '進', '盤踞', '進入', '去

|         Type        |     Description    |
|:-------------------:|:------------------:|
| A                   | 非謂形容詞         |
| Caa                 | 對等連接詞         |
| Cab                 | 連接詞，如：等等   |
| Cba                 | 連接詞，如：的話   |
| Cbb                 | 關聯連接詞         |
| D                   | 副詞               |
| Da                  | 數量副詞           |
| Dfa                 | 動詞前程度副詞     |
| Dfb                 | 動詞後程度副詞     |
| Di                  | 時態標記           |
| Dk                  | 句副詞             |
| DM                  | 定量式             |
| I                   | 感嘆詞             |
| Na                  | 普通名詞           |
| Nb                  | 專有名詞           |
| Nc                  | 地方詞             |
| Ncd                 | 位置詞             |
| Nd                  | 時間詞             |
| Nep                 | 指代定詞           |
| Neqa                | 數量定詞           |
| Neqb                | 後置數量定詞       |
| Nes                 | 特指定詞           |
| Neu                 | 數詞定詞           |
| Nf                  | 量詞               |
| Ng                  | 後置詞             |
| Nh                  | 代名詞             |
| Nv                  | 名物化動詞         |
| P                   | 介詞               |
| T                   | 語助詞             |
| VA                  | 動作不及物動詞     |
| VAC                 | 動作使動動詞       |
| VB                  | 動作類及物動詞     |
| VC                  | 動作及物動詞       |
| VCL                 | 動作接地方賓語動詞 |
| VD                  | 雙賓動詞           |
| VF                  | 動作謂賓動詞       |
| VE                  | 動作句賓動詞       |
| VG                  | 分類動詞           |
| VH                  | 狀態不及物動詞     |
| VHC                 | 狀態使動動詞       |
| VI                  | 狀態類及物動詞     |
| VJ                  | 狀態及物動詞       |
| VK                  | 狀態句賓動詞       |
| VL                  | 狀態謂賓動詞       |
| V_2                 | 有                 |
|                     |                    |
| DE                  | 的之得地           |
| SHI                 | 是                 |
| FW                  | 外文               |
|                     |                    |
| COLONCATEGORY       | 冒號               |
| COMMACATEGORY       | 逗號               |
| DASHCATEGORY        | 破折號             |
| DOTCATEGORY         | 點號               |
| ETCCATEGORY         | 刪節號             |
| EXCLAMATIONCATEGORY | 驚嘆號             |
| PARENTHESISCATEGORY | 括號               |
| PAUSECATEGORY       | 頓號               |
| PERIODCATEGORY      | 句號               |
| QUESTIONCATEGORY    | 問號               |
| SEMICOLONCATEGORY   | 分號               |
| SPCHANGECATEGORY    | 雙直線             |
| WHITESPACE          | 空白               |

In [31]:
def get_bow_vector_with_selection(pairs, word2index):
    excluded_flags = [
        # 根據以上列舉出來的文字以及詞性表，請列出想要排除的詞性
        # YOUR CODE HERE
        'COLONCATEGORY', 'COMMACATEGORY', 'DASHCATEGORY', 'DOTCATEGORY', 'ETCCATEGORY', 'EXCLAMATIONCATEGORY', 
        'PARENTHESISCATEGORY', 'PAUSECATEGORY', 'PERIODCATEGORY','SEMICOLONCATEGORY', 'SPCHANGECATEGORY', 'WHITESPACE',
        'QUESTIONCATEGORY', 'V_2', 'DE', 'SHI', 'I', 'T'
        # END YOUR CODE
    ]
    vector = np.zeros(len(word2index))
    for word, flag in pairs:
        if word in word2index and flag not in excluded_flags:
            vector[word2index[word]] += 1
    return vector

# Cosine Similarity

In [27]:
def cosine_similarity(bow1, bow2):
    # YOUR CODE HERE
    similarity = np.dot(bow1, bow2)/(np.linalg.norm(bow1)*np.linalg.norm(bow2))

    # END YOUR CODE
    return similarity

In [28]:
bow1 = get_bow_vector(train_title_cuts[100], word2index)
bow2 = get_bow_vector(train_title_cuts[130], word2index)
cosine_similarity(bow1, bow2)

0.08703882797784893

In [29]:
train_title_cuts[100]

[('山東', 'Nc'),
 ('魯能', 'Nb'),
 ('有沒有', 'D'),
 ('可能', 'D'),
 ('拿到', 'VC'),
 ('今年', 'Nd'),
 ('的', 'DE'),
 ('中', 'A'),
 ('超', 'A'),
 ('冠軍', 'Na'),
 ('？', 'QUESTIONCATEGORY')]

In [30]:
train_title_cuts[130]

[('NBA', 'Nb'),
 ('和', 'Caa'),
 ('CBA', 'FW'),
 ('差距', 'Na'),
 ('在', 'P'),
 ('哪裡', 'Ncd'),
 ('？', 'QUESTIONCATEGORY'),
 ('6', 'Neu'),
 ('張', 'Nf'),
 ('圖', 'VF'),
 ('一目瞭然', 'VH'),
 ('！', 'EXCLAMATIONCATEGORY')]

# Group mean vector

In [32]:
group_vectors = {news_class: [] for news_class in all_news_class}
for index, pairs in sorted(train_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    news_class = train_classes[index]
    group_vectors[news_class].append(vector)

group_mean_vector = {}
for news_class, vectors in group_vectors.items():
    group_mean_vector[news_class] = np.mean(vectors, axis=0)
group_mean_vector

{'體育': array([0.04      , 0.00333333, 0.        , ..., 0.        , 0.        ,
        0.        ]),
 '財經': array([0., 0., 0., ..., 0., 0., 0.]),
 '科技': array([0., 0., 0., ..., 0., 0., 0.]),
 '旅遊': array([0., 0., 0., ..., 0., 0., 0.]),
 '農業': array([0., 0., 0., ..., 0., 0., 0.]),
 '遊戲': array([0.        , 0.        , 0.        , ..., 0.00333333, 0.00333333,
        0.00333333])}

In [33]:
# group_vectors = {news_class: [] for news_class in all_news_class}
# for index, pairs in sorted(train_title_cuts.items()):
#     vector = get_bow_vector_with_selection(pairs, word2index)
#     news_class = train_classes[index]
#     group_vectors[news_class].append(vector)

# group_mean_vector = {}
# for news_class, vectors in group_vectors.items():
#     group_mean_vector[news_class] = np.mean(vectors, axis=0)
# group_mean_vector

# Group mean vector: 測試

In [34]:
classification = {news_class: [] for news_class in all_news_class}
for index, pairs in sorted(test_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    if np.sum(np.square(vector)) == 0:
        continue

    max_val = -2.0
    max_class = None
    for news_class, ref_vector in group_mean_vector.items():
        val = cosine_similarity(ref_vector, vector)
        if val > max_val:
            max_class = news_class
            max_val = val

    classification[max_class].append(index)

In [35]:
from collections import Counter

for group, ids in classification.items():
    counter = Counter([test_classes[id] for id in ids])
    print('predict', group, ': ', counter)

predict 體育 :  Counter({'體育': 72, '遊戲': 10, '農業': 6, '財經': 5, '旅遊': 3, '科技': 2})
predict 財經 :  Counter({'財經': 65, '科技': 21, '農業': 11, '遊戲': 6, '體育': 5, '旅遊': 5})
predict 科技 :  Counter({'科技': 61, '財經': 16, '體育': 9, '農業': 9, '旅遊': 8, '遊戲': 6})
predict 旅遊 :  Counter({'旅遊': 68, '農業': 8, '財經': 6, '體育': 2, '科技': 1, '遊戲': 1})
predict 農業 :  Counter({'農業': 61, '旅遊': 6, '科技': 4, '體育': 3, '財經': 3, '遊戲': 1})
predict 遊戲 :  Counter({'遊戲': 76, '科技': 10, '旅遊': 10, '體育': 8, '財經': 5, '農業': 4})
